In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 199 kB 39.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=47f35faff8734f7ca9505390469d1821f5e397b74b695fc6dc82cce8c263e9df
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

In [ ]:
df = spark.read.csv("/content/googleplaystore.csv", header=True)
df.printSchema()
df.show(5)


root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+--------------

In [ ]:
df.select("App", "Rating", "Last Updated") .show(5)

+--------------------+------+----------------+
|                 App|Rating|    Last Updated|
+--------------------+------+----------------+
|Photo Editor & Ca...|   4.1| January 7, 2018|
| Coloring book moana|   3.9|January 15, 2018|
|U Launcher Lite –...|   4.7|  August 1, 2018|
|Sketch - Draw & P...|   4.5|    June 8, 2018|
|Pixel Draw - Numb...|   4.3|   June 20, 2018|
+--------------------+------+----------------+
only showing top 5 rows



In [ ]:
df_filtered_1 = df.filter("Rating > 4.0 AND Type == 'Free'")

df_filtered_1 .show(5)
df_filtered_1 .count()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+---------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|   Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+---------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|January 7, 2018|             1.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design| August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|    0|          Teen|        Art & Design|   June 8, 2018|Varies with device|  4.2 and up|
|Pixel Dra

7614

In [ ]:
from pyspark.sql import functions as F
df_filtered_2 = df.filter(
    (F.col("Type") == "Free") &
    (F.col("Rating") > 4.0)
)

df_filtered_2 .show(5)
df_filtered_2 .count()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+---------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|   Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+---------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|January 7, 2018|             1.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design| August 1, 2018|             1.2.4|4.0.3 and up|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50,000,000+|Free|    0|          Teen|        Art & Design|   June 8, 2018|Varies with device|  4.2 and up|
|Pixel Dra

7614

In [ ]:
df = df.withColumn("Reviews_k", F.col("Reviews") / 1000)
df.select("Reviews_K") .show(5)

df.printSchema()

+---------+
|Reviews_K|
+---------+
|    0.159|
|    0.967|
|    87.51|
|  215.644|
|    0.967|
+---------+
only showing top 5 rows

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)
 |-- Reviews_k: double (nullable = true)



In [ ]:
df.groupby("Category") .agg(
    F.mean("Rating") .alias("Avg_Rating")
) .show()

+-------------------+-----------------+
|           Category|       Avg_Rating|
+-------------------+-----------------+
|             EVENTS|              NaN|
|             COMICS|              NaN|
|             SPORTS|              NaN|
|            WEATHER|              NaN|
|      VIDEO_PLAYERS|              NaN|
|  AUTO_AND_VEHICLES|              NaN|
|          PARENTING|              NaN|
|      ENTERTAINMENT|4.126174496644294|
|    PERSONALIZATION|              NaN|
| HEALTH_AND_FITNESS|              NaN|
|   TRAVEL_AND_LOCAL|              NaN|
|BOOKS_AND_REFERENCE|              NaN|
|     FOOD_AND_DRINK|              NaN|
|        PHOTOGRAPHY|              NaN|
|           BUSINESS|              NaN|
|             FAMILY|              NaN|
|           SHOPPING|              NaN|
|     HOUSE_AND_HOME|              NaN|
|               GAME|              NaN|
|               Face|             null|
+-------------------+-----------------+
only showing top 20 rows



In [ ]:
df.filter("Rating <> 'NaN' and Rating is not null") .groupby ("Category") .agg(
    F.mean("Rating") .alias("Avg_Rating")
).show()

+-------------------+------------------+
|           Category|        Avg_Rating|
+-------------------+------------------+
|             EVENTS| 4.435555555555557|
|             COMICS| 4.155172413793104|
|             SPORTS| 4.223510971786835|
|            WEATHER| 4.243999999999999|
|      VIDEO_PLAYERS| 4.063750000000001|
|  AUTO_AND_VEHICLES|  4.19041095890411|
|          PARENTING| 4.300000000000001|
|      ENTERTAINMENT| 4.126174496644294|
|    PERSONALIZATION| 4.335987261146501|
| HEALTH_AND_FITNESS|4.2773648648648654|
|   TRAVEL_AND_LOCAL|  4.10929203539823|
|BOOKS_AND_REFERENCE| 4.346067415730338|
|     FOOD_AND_DRINK|4.1669724770642205|
|        PHOTOGRAPHY| 4.192113564668767|
|           BUSINESS| 4.121452145214522|
|             FAMILY| 4.192272467086437|
|           SHOPPING| 4.259663865546221|
|     HOUSE_AND_HOME| 4.197368421052633|
|               GAME|4.2863263445761195|
|               Face|              null|
+-------------------+------------------+
only showing top

In [ ]:
df = spark.read.csv("/content/googleplaystore_user_reviews.csv", header=True)
df.printSchema()
df.show(5)

root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)

+--------------------+--------------------+--------------------+------------------+----------------------+
|                 App|   Translated_Review|           Sentiment|Sentiment_Polarity|Sentiment_Subjectivity|
+--------------------+--------------------+--------------------+------------------+----------------------+
|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|          Positive|                   1.0|
|10 Best Foods for...|This help eating ...|            Positive|              0.25|   0.28846153846153844|
|10 Best Foods for...|                 nan|                 nan|               nan|                   nan|
|10 Best Foods for...|Works great espec...|            Positive|               0.4|                 0.875|
|10 Best Foods f

In [ ]:
df.join(df_reviews,
        df.App == df_reviews .App, 
        "fullouter")
.show()

NameError: ignored